IMPORTS

In [73]:
# Imports
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, asdict
from typing import Any, Dict, List, Optional, Tuple
from deap import base, creator, tools

# File name
FILENAME = "tsp.dat"

HELPER FUNCTIONS

In [74]:
def haversine(lat_p1, long_p1, lat_p2, long_p2):
    """
    Function calculates the haversine distance between 2 points.
    Refer to this website regarding calculation: https://en.wikipedia.org/wiki/Haversine_formula
    """
    Earth_radius = 3958.88      # miles
    # Convert to radian for math. functions
    phi_1 = math.radians(lat_p1)
    phi_2 = math.radians(lat_p2)
    lambda_p1 = math.radians(long_p1)
    lambda_p2 = math.radians(long_p2)

    dphi = phi_2 - phi_1
    dlambda = lambda_p2 - lambda_p1

    hav_dphi = math.pow(math.sin(dphi/2), 2)
    cos_phi1 = math.cos(phi_1)
    cos_phi2 = math.cos(phi_2)
    hav_dlambda = math.pow(math.sin(dlambda/2), 2)

    hav_theta = hav_dphi + cos_phi1*cos_phi2*hav_dlambda
    theta = 2 * math.asin(math.sqrt(hav_theta))

    return theta * Earth_radius

def haversine_matrix(coordinates):
    """
    This function will calculate the haversine distance between all pair of coordinates, that way eval(ind) will not need to
    coordinates: k points of {latitude, longitude}
    """
    k = len(coordinates)

    # Create the K*K matrix and init with 0s
    hav_matrix = np.zeros((k, k))

    # compute the haversine distance for each k
    for i in range(k):
        lat_1, long_1 = coordinates[i]      # CityPoint 1

        for j in range(i + 1, k):
            lat_2, long_2 = coordinates[j]      # CityPoint 2 through k

            haversine_distance = haversine(lat_1, long_1, lat_2, long_2)

            # based off calc, matrix should be symmetric i.e D[i][j] = D[j][i]
            # Diagional i.e D[i][i] always = 0
            # Ex: Distance from NYC to Paris == Distance from Paris to NYC
            hav_matrix[i][j] = haversine_distance
            hav_matrix[j][i] = haversine_distance

    return hav_matrix

def evaluate_TSP(Haversine_Dist_matrix):
    """
    We need to take in the haversine_matrix as it holds the precomputed haversine distances, 
    from here we evaluate the permutations of city-point indices.
    We then need to walk through the list of the permutation, 
    adding the distances from the current city to the adjacent city.
    """
    def eval(ind):
        total_dist = 0.0

        # Walk through the permutation
        for i in range(len(ind) - 1):
            cur_city = ind[i]
            adjacent_city = ind[i + 1]

            total_dist += Haversine_Dist_matrix[cur_city][adjacent_city]
        
        # Complete the cycle
        total_dist += Haversine_Dist_matrix[ind[-1]][ind[0]]
        return (total_dist,)  # DEAP expects tuple
        
    return eval

def compare_with_best(best_min_distance):
    # Best minimum distance is 10,637.36 miles
    optimum = 10637.36
    # Compare our best with the true best and represent as a %
    percent = ((best_min_distance - optimum) / optimum ) * 100
    return percent

def load_dat(file):
    cities = []
    # Latitude Longitude Coordinates
    coordinates = []

    with open(file, "r") as f:
        for capital_cities in f:
            # Extract information
            parts = capital_cities.strip().split()

            # FORMAT: Albany, NY          42.652552778 -73.75732222
            city_state = " ".join(parts[:-2]) 
            latitude = float(parts[-2])
            longitude = float(parts[-1])

            # Append to list
            cities.append(city_state)
            coordinates.append((latitude, longitude))

    return cities, coordinates



FUNCTION TESTER

In [75]:
# cities, coordinates = load_dat(FILENAME)
# print(haversine_matrix(coordinates))   # Verified haversine distance for first 3 coordinates 

Genetic Algorithm (GA) Configuration and Implementation

In [76]:
# DEAP setup
def ensure_deap_creators() -> None:
    if not hasattr(creator, "FitnessMin"):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))     # We are MINIMIZING
    if not hasattr(creator, "Individual"):
        creator.create("Individual", list, fitness=creator.FitnessMin)


def mutInsert(individual):
    size = len(individual)
    i, j = sorted(random.sample(range(size), 2))

    gene = individual.pop(j)
    individual.insert(i, gene)

    return (individual,)
        
# ===================== GA Configuration Dataclass =====================
# Configuration dataclass
@dataclass(frozen=True)         # Parameters cant be changed during runs
class Config:
    genome_length: int = 49     # 49 indices of city, cooridnates
    pop_size: int = 1500        # n individuals
    generations: int = 1000     # Allowed generations
    cxpb: float = 0.7           # crossover prob
    mutpb: float = 0.19         # mutation probability
    tournsize: int = 3          # For tournament slection
    eliteSize: int = 1          # Best k individuals to transfer to next gen
    seed: Optional[int] = None

# Build DEAP toolbox
def build_toolbox(cfg: Config) -> base.Toolbox:
    ensure_deap_creators()

    toolbox = base.Toolbox()
    # EX of ind: [1, 2, 3, 4......49]  representing city1 --> city2-->city49
    toolbox.register("indices", random.sample, range(cfg.genome_length), cfg.genome_length) # Creates the list of indices reppresenting cities 1-49
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", tools.mutInversion)
    toolbox.register("select", tools.selTournament, tournsize=cfg.tournsize)

    return toolbox


# Single run of the GA
def run(cfg: Config) -> Dict[str, Any]:
    if cfg.seed is not None:
        random.seed(cfg.seed)
        np.random.seed(cfg.seed)

    # List of city names and coordinates
    cities, coordinates = load_dat(FILENAME)

    # Fill the Haversine matrix with the Haversine distances for each City-Point
    Haversine_Dist_matrix = haversine_matrix(coordinates)

    # Setup toolbox
    toolbox = build_toolbox(cfg)
    toolbox.register("evaluate", evaluate_TSP(Haversine_Dist_matrix))
    pop = toolbox.population(n=cfg.pop_size)    # Create initial popultation

    # Evaluate initial population (gen=0)
    fitnesses = map(toolbox.evaluate, pop)  # Compute fitness for each ind
    for ind, fit in zip(pop, fitnesses):    # Assign fitness back to ind
        ind.fitness.values = fit

    # Records Global Best solution, so that it is not lost
    hall_of_fame = tools.HallOfFame(cfg.eliteSize)
    
    # Link current statistical information we want for each indivudal
    statistics = tools.Statistics(lambda ind: ind.fitness.values[0])    # Get each ind's fitness value
    statistics.register("min", np.min)
    statistics.register("mean", np.mean)
    statistics.register("std", np.std)

    # Make a logbook for future reference
    logbook = tools.Logbook()
    logbook.header = ("gen", "nevals", "mean", "min", "std")

    # Fill in logbook w/ initial info
    stats = statistics.compile(pop)
    logbook.record(gen=0, nevals=len(pop), **stats)

    hall_of_fame.update(pop)
    # Evolution loop
    for gen in range(1, cfg.generations + 1):
        # Select offspring
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cfg.cxpb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation
        for mutant in offspring:
            if random.random() < cfg.mutpb:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate individuals with invalid fitness
        invalid = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid)      # Produce new fitness values
        for ind, fit in zip(invalid, fitnesses):
            ind.fitness.values = fit

        # Elitism update if any new global best made from offspring
        hall_of_fame.update(offspring)

        # Inject hall of fame ind back into population
        offspring.sort(key=lambda ind: ind.fitness.values[0])
        for ind in range(cfg.eliteSize):
            offspring[ind] = toolbox.clone(hall_of_fame[ind])     
        
        pop[:] = offspring

        # Update Logbook for current generation
        stats = statistics.compile(pop)
        logbook.record(gen=gen, nevals=len(pop), **stats)

        # For console printing
        print(logbook.stream)
    
    best_ind = hall_of_fame[0]
    best_distance = best_ind.fitness.values[0]

    # Convert the indices into their city names
    tour_named = [cities[i] for i in best_ind]
    # Print start point for clarity
    tour_named.append(cities[best_ind[0]])

    return {
        "best_distance": best_distance,
        "best_tour_indices": best_ind,
        "tour_cities": tour_named,
        "logbook": logbook
    }
        

MAIN

In [77]:
def main():
    cfg = Config(seed=18)
    results = run(cfg)

    logbook = results["logbook"]    # access the logbook
    best_distance = results["best_distance"]
    best_ind = results["best_tour_indices"]
    cities_toured = results["tour_cities"]
    accuracy = compare_with_best(best_distance)
    print(logbook)

    print("\n===========================================")
    print("SHORTES PATH TOUR OBTAINED")
    print("===========================================")
    print(f"Best Minimum Distance Found = {best_distance:.4f} miles\n")
    print(f"Percentage difference from optimum solution: {accuracy:.4f}")
    
    print("Best Tour (Indices) Found:", best_ind)
    print("Best Tour (City names) Found:", cities_toured)



In [78]:
if __name__ == "__main__":
    main()

gen	nevals	mean   	min    	std    
0  	1500  	50649.3	38431  	3507.1 
1  	1500  	48056.4	36288.4	3156.43
2  	1500  	46181.9	36288.4	2921.75
3  	1500  	44499.8	35828  	2852.41
4  	1500  	43062.8	35499.6	2773.32
5  	1500  	41938.7	32418.3	2810.78
6  	1500  	40895  	29958.1	2776.84
7  	1500  	39826.4	29958.1	2894.01
8  	1500  	38766.3	28231.6	2705.89
9  	1500  	38005.5	28231.6	2761.74
10 	1500  	37260.8	27210.1	2793.73
11 	1500  	36371.4	27210.1	2709.29
12 	1500  	35546.5	27210.1	2722.29
13 	1500  	34856.8	27210.1	2637.41
14 	1500  	34300.1	27210.1	2770.11
15 	1500  	33561  	27210.1	2726.61
16 	1500  	32759.9	26311.2	2582.7 
17 	1500  	32181.1	25773.1	2454.36
18 	1500  	31702.8	24649.3	2451.75
19 	1500  	31130.4	24262.4	2462.45
20 	1500  	30659.2	24262.4	2394.44
21 	1500  	30215.2	24262.4	2300.57
22 	1500  	29985.6	23770.5	2411.89
23 	1500  	29456.9	22986.9	2321.34
24 	1500  	29144.7	22901.3	2407.01
25 	1500  	28682.4	18742.1	2314.91
26 	1500  	28396.2	18742.1	2374.84
27 	1500  	27905.2	1